In [30]:
from supabase import create_client
import dotenv
import os
import pandas as pd
from PIL import Image
from io import BytesIO

dotenv.load_dotenv()

True

In [3]:
supabase = create_client(os.getenv('EXPO_PUBLIC_SUPABASE_URL'), os.getenv('EXPO_PUBLIC_SUPABASE_ANON_KEY'))

In [83]:
responseList = supabase.storage.from_("members").list(options={'limit': 200})

In [84]:
# Check if files are found
if responseList:
    # Process files into a list of dictionaries
    file_data = [
        {
            "name": file["name"],
            "type": os.path.splitext(file["name"])[1].lstrip("."),  # Extract file extension (type)
            "size": file["metadata"]["size"] / 1024  # Convert size to KB
        }
        for file in responseList
    ]

    # Convert to DataFrame
    df = pd.DataFrame(file_data)

    # Sort: Files > 100KB first
    df = df.sort_values(by="size", ascending=False)
else:
    print("No files found in the 'members' storage.")

In [85]:
df['type'] = df['type'].str.lower()

In [86]:
df[(df['size'] > 100) | (df['type'] != 'png') ].shape

(48, 3)

In [87]:
def convert_to_png(image_bytes):
    with Image.open(BytesIO(image_bytes)) as img:
        output = BytesIO()
        img.save(output, format="PNG")
        output.seek(0)
        return output.getvalue()

    return image_bytes  # Return as is if PNG

In [88]:
def scale_down_image(image_bytes, max_size):
    with Image.open(BytesIO(image_bytes)) as img:
        output = BytesIO()

        # Reduce quality until under 100KB
        quality = 95  # Start with high quality
        while output.tell() / 1024 > max_size and quality > 10:
            output = BytesIO()
            img.save(output, format="PNG", quality=quality)
            output.seek(0)
            quality -= 5

        return output.getvalue()

In [101]:
file_bytes = supabase.storage.from_("members").download("IMG_5141 - amargirish (2).jpeg")

In [102]:
file_bytes = convert_to_png(file_bytes)

In [103]:
image = Image.open(BytesIO(file_bytes))

In [ ]:
image.resize((500, 500), Image.ANTIALIAS)

In [104]:
image.size

(2316, 2030)

In [99]:
file_bytes = scale_down_image(file_bytes, 100)

In [100]:
new_file_name = "IMG_5141 - amargirish (2).png"
supabase.storage.from_("members").upload(file=file_bytes, path=new_file_name, file_options={'upsert': "true"})

UploadResponse(path='IMG_5141 - amargirish (2).png', full_path='members/IMG_5141 - amargirish (2).png')

In [95]:
for image in df[(df['size'] > 100) | (df['type'] != 'png') ].to_dict(orient='records'):
    file_bytes = supabase.storage.from_("members").download(image['name'])
    
    try:
        if image['type'] != 'png':
            print('Not png: ', image['name'])
            file_bytes = convert_to_png(file_bytes)
        
        if image['size'] > 100 and image['size'] < 2000:
            file_bytes = scale_down_image(file_bytes, 100)
        
        new_file_name = os.path.splitext(image['name'])[0] + ".png"
        supabase.storage.from_("members").upload(file=file_bytes, path=new_file_name, file_options={'upsert': "true"})
    except Exception as e:
        print(e)
        print(image)

Not png:  IMG_5141 - amargirish (2).jpeg
Not png:  fheQk10hvUgStwsZSGN6LcYz94N2.jpg
Not png:  f4Xuy2P6qBbKnMnwEVg3sPLv28i2.jpg
Not png:  Venkataraman - Fin 1 Solutions.jpeg
Not png:  q3ZmFxvjshXG7Zx0DzNITbiH90q1.JPG
Not png:  Karthikeyan Photo - Money Time Secutiry.jpg
Not png:  IMG-20241213-WA0039 - anuradha Srikanth.jpg
Not png:  WhatsApp Image 2025-01-11 at 21.06.21_e9812187 - Tarun Gopinath.jpg
Not png:  H1OEHEHxMzdIbhmMOjUdUYJ6cRr2.jpg
Not png:  bU62C8QMwMQ8pHSYJHeT29Fjr8H2.jpg
Not png:  IMG-20241106-WA0023(1) - Niky pedia.jpg
Not png:  GNhogVtO49Nw2Y9le1lFQpRcrAr2.jpg
Not png:  gY6LtyHpGQaugLxsMDEy86H7yGy2.jpg
Not png:  sudha_sathiyan.jpg
Not png:  karthick.jpg
Not png:  88IZ12up0kYsw9jbyvLaMZ3qjOm1.jpg
Not png:  48ywCUWkqnP7IwVM3StBztjZ1Ar1.JPG
Not png:  0pVjtsdPn9fQE0IylQKcFqmdVQJ3.jpg
Not png:  20250107_215757 - Keerthi Girish.jpg
Not png:  hMgLtuNVZiMJj39xzfk5HRryBZC3.jpg
Not png:  01 - Ajay tata (1).jpg
Not png:  Pinakee coat - Pinakee Roy.jpeg
Not png:  035A8299 - Manish Ku

In [90]:
responseListAfter = supabase.storage.from_("members").list(options={'limit': 200})

In [92]:
# Check if files are found
if responseListAfter:
    # Process files into a list of dictionaries
    file_data_after = [
        {
            "name": file["name"],
            "type": os.path.splitext(file["name"])[1].lstrip("."),  # Extract file extension (type)
            "size": file["metadata"]["size"] / 1024  # Convert size to KB
        }
        for file in responseListAfter
    ]

    # Convert to DataFrame
    df_after = pd.DataFrame(file_data_after)

    # Sort: Files > 100KB first
    df_after = df_after.sort_values(by="size", ascending=False)
else:
    print("No files found in the 'members' storage.")

In [93]:
df_after[df_after['size']> 100]

,name,type,size
112,VcklLLsmlaPvJGSJJ9oe14puJaQ2.png,png,9018.547852
123,YpNXPZOjQ5Rdxth0N9g7ab3pH1i1.png,png,6440.762695
74,lPV9dg813paS5cZlYpMlLCXsDXs1.png,png,6314.470703
128,zDYKY9HF3SYHtlTTaH4UxgNVSHQ2.png,png,6102.082031
42,FzIjbeoygQRt4spAJalnEqNd0Pp1.png,png,6048.000000
83,oknv5vP2wAZKVO94zxHoKD1ZYSJ3.png,png,5725.393555
82,oh4CqM27M4PVEJLKj8uFVigMYEJ2.png,png,2495.465820
48,gY6LtyHpGQaugLxsMDEy86H7yGy2.png,png,1754.725586
45,GNhogVtO49Nw2Y9le1lFQpRcrAr2.png,png,1409.098633
57,IMG-20241106-WA0023(1) - Niky pedia.png,png,1127.920898
